In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
from textrec.paths import paths

In [3]:
experiments = ['gc1', 'spec1']
frames = [pd.read_csv(paths.analyzed / f'trial_withmanual_{exp}.csv') for exp in experiments]
# Approach based on https://stackoverflow.com/a/48064892/69707
columns_ordered = []
for frame in frames:
    for col in frame.columns:
        if col not in columns_ordered:
            columns_ordered.append(col)
all_data = pd.concat(
    frames,
    keys=experiments,
    names=('experiment', '_idx'))[columns_ordered].reset_index(level=0).reset_index(drop=True)

In [4]:
all_data['condition_mapped'] = all_data.condition.map(lambda x: {'always': 'general'}.get(x,x))

In [5]:
all_data['condition_order_mapped'] = all_data.condition_order.map(lambda x: x.replace('always', 'general'))

In [6]:
norecs_speed = all_data[all_data.condition == 'norecs'].groupby('participant').characters_per_sec.mean()
with_norecs_speed = pd.merge(
    all_data,
    norecs_speed.to_frame('chars_per_sec_norecs_mean'),
    left_on='participant',
    right_index=True)
with_norecs_speed

,experiment,participant,age,english_proficiency,gender,helpfulRank-accurate-least-condition,helpfulRank-accurate-least-idx,helpfulRank-accurate-most-condition,helpfulRank-accurate-most-idx,helpfulRank-quick-least-condition,...,corrected_idealrecuse_always,corrected_bow_recs_offered_always,corrected_bow_recs_idealuse_always,corrected_tapstotype_cond,corrected_efficiency,rec_appropriation_rate,rec_appropriation_rate_boxcox,condition_mapped,condition_order_mapped,chars_per_sec_norecs_mean
0,gc1,26f62q,33.0,Native or bilingual,male,norecs,0,always,2,norecs,...,13,13,7,57,0.311475,NaN,NaN,norecs,"norecs,gated,general",2.118500
1,gc1,26f62q,33.0,Native or bilingual,male,norecs,0,always,2,norecs,...,13,13,7,73,0.820225,NaN,NaN,norecs,"norecs,gated,general",2.118500
2,gc1,26f62q,33.0,Native or bilingual,male,norecs,0,always,2,norecs,...,13,13,9,69,0.896104,NaN,NaN,norecs,"norecs,gated,general",2.118500
3,gc1,26f62q,33.0,Native or bilingual,male,norecs,0,always,2,norecs,...,12,12,10,72,1.000000,NaN,NaN,norecs,"norecs,gated,general",2.118500
4,gc1,26f62q,33.0,Native or bilingual,male,norecs,0,always,2,norecs,...,10,10,6,17,0.425000,NaN,NaN,gated,"norecs,gated,general",2.118500
5,gc1,26f62q,33.0,Native or bilingual,male,norecs,0,always,2,norecs,...,13,13,6,28,0.341463,NaN,NaN,gated,"norecs,gated,general",2.118500
6,gc1,26f62q,33.0,Native or bilingual,male,norecs,0,always,2,norecs,...,19,19,10,38,0.330435,NaN,NaN,gated,"norecs,gated,general",2.118500
7,gc1,26f62q,33.0,Native or bilingual,male,norecs,0,always,2,norecs,...,15,15,10,25,0.378788,NaN,NaN,gated,"norecs,gated,general",2.118500
8,gc1,26f62q,33.0,Native or bilingual,male,norecs,0,always,2,norecs,...,9,9,5,16,0.457143,NaN,NaN,general,"norecs,gated,general",2.118500
9,gc1,26f62q,33.0,Native or bilingual,male,norecs,0,always,2,norecs,...,11,11,6,19,0.475000,NaN,NaN,general,"norecs,gated,general",2.118500


In [7]:
with_norecs_speed['chars_per_sec_ratio_to_norecs'] = with_norecs_speed.characters_per_sec / with_norecs_speed.chars_per_sec_norecs_mean

In [8]:
with_norecs_speed['chars_per_sec_ratio_to_norecs_log'] = np.log(with_norecs_speed.chars_per_sec_ratio_to_norecs)

In [9]:
import scipy.stats

In [10]:
with_norecs_speed['chars_per_sec_ratio_to_norecs_boxcox'], boxcox_lambda = scipy.stats.boxcox(with_norecs_speed.chars_per_sec_ratio_to_norecs)

In [11]:
with_norecs_speed.to_csv(paths.analyzed / 'combined_traits.csv', index=False)